# Sentinel-2 Data Extraction for Algerian Crop Samples (2023)

This notebook extracts Sentinel-2 imagery from Google Earth Engine for the Algerian field samples.

**Parameters:**
- **Year:** 2023 (from XML metadata: CreaDate=20230126)
- **Time Range:** January - April 2023 (growing season for cereals)
- **Bands:** 10 bands matching PASTIS format (B2, B3, B4, B5, B6, B7, B8, B8A, B11, B12)
- **Patch Size:** 128×128 pixels at 10m resolution
- **Output Format:** NumPy arrays (.npy) matching PASTIS structure

In [1]:
# Install required packages
import subprocess
import sys

# Install geemap if not installed
try:
    import geemap
    print("geemap already installed")
except ImportError:
    print("Installing geemap...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "geemap", "-q"])
    import geemap
    print("geemap installed successfully")

# Install earthengine-api if not installed
try:
    import ee
    print("earthengine-api already installed")
except ImportError:
    print("Installing earthengine-api...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "earthengine-api", "-q"])
    import ee
    print("earthengine-api installed successfully")

geemap already installed
earthengine-api already installed


In [3]:
# Authenticate and Initialize Google Earth Engine
import ee
import geemap

# Initialize with project ID
try:
    ee.Initialize(project='mythical-sweep-471412-h5')
    print("GEE initialized successfully!")
except Exception as e:
    print("Authenticating with Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project='mythical-sweep-471412-h5')
    print("GEE authenticated and initialized successfully!")

GEE initialized successfully!


In [ ]:
# Load shapefiles and prepare data
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Define paths
BASE_DIR = Path('/home/crop/Desktop/crop2')
SAMPLES_DIR = BASE_DIR / 'Samples_cereal_In-situ Algerian' / 'Samples_cereal'
OUTPUT_DIR = BASE_DIR / 'output' / 'algeria_s2_data'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load all shapefiles
samples_cereal = gpd.read_file(SAMPLES_DIR / 'samples_cereal.shp')
samples_non_cereal = gpd.read_file(SAMPLES_DIR / 'samples_non_cereal.shp')
validation_cereal = gpd.read_file(SAMPLES_DIR / 'validation_cereal.shp')
validation_non_cereal = gpd.read_file(SAMPLES_DIR / 'validation_non_cereal.shp')

# Add class labels
samples_cereal['class'] = 1  # cereal
samples_non_cereal['class'] = 0  # non-cereal
validation_cereal['class'] = 1
validation_non_cereal['class'] = 0

# Add dataset type
samples_cereal['split'] = 'train'
samples_non_cereal['split'] = 'train'
validation_cereal['split'] = 'validation'
validation_non_cereal['split'] = 'validation'

# Combine all samples
all_samples = gpd.GeoDataFrame(
    pd.concat([samples_cereal, samples_non_cereal, validation_cereal, validation_non_cereal], ignore_index=True),
    crs=samples_cereal.crs
)

# Add unique ID
all_samples['patch_id'] = range(len(all_samples))

print(f"Total samples: {len(all_samples)}")
print(f"  - Training cereal: {len(samples_cereal)}")
print(f"  - Training non-cereal: {len(samples_non_cereal)}")
print(f"  - Validation cereal: {len(validation_cereal)}")
print(f"  - Validation non-cereal: {len(validation_non_cereal)}")
print(f"\nBounds: {all_samples.total_bounds}")
print(f"CRS: {all_samples.crs}")

Total samples: 117
  - Training cereal: 1
  - Training non-cereal: 62
  - Validation cereal: 1
  - Validation non-cereal: 53

Bounds: [-0.86667757 34.96964575 -0.53314745 35.25219505]
CRS: EPSG:4326


In [4]:
# Define Sentinel-2 data extraction parameters
import pandas as pd

# Time range for growing season (Jan-April 2023)
START_DATE = '2023-01-01'
END_DATE = '2023-04-30'

# Bands to extract (matching PASTIS format)
S2_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

# Patch size
PATCH_SIZE = 128  # 128x128 pixels at 10m = 1.28km x 1.28km

# Resolution (meters)
RESOLUTION = 10

print(f"Extraction Parameters:")
print(f"  - Date Range: {START_DATE} to {END_DATE}")
print(f"  - Bands: {S2_BANDS}")
print(f"  - Patch Size: {PATCH_SIZE}x{PATCH_SIZE} pixels")
print(f"  - Resolution: {RESOLUTION}m")
print(f"  - Spatial extent: {PATCH_SIZE * RESOLUTION / 1000:.2f}km x {PATCH_SIZE * RESOLUTION / 1000:.2f}km per patch")

Extraction Parameters:
  - Date Range: 2023-01-01 to 2023-04-30
  - Bands: ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
  - Patch Size: 128x128 pixels
  - Resolution: 10m
  - Spatial extent: 1.28km x 1.28km per patch


In [5]:
# Functions to create 128x128 patches from polygons
def get_patch_bounds(centroid_x, centroid_y, patch_size=128, resolution=10):
    """Calculate patch bounds centered on a point"""
    half_extent = (patch_size * resolution) / 2  # meters
    # Convert degrees to meters (approximate for Algeria ~35°N)
    deg_per_meter_lon = 1 / (111320 * np.cos(np.radians(centroid_y)))
    deg_per_meter_lat = 1 / 110540
    
    half_extent_lon = half_extent * deg_per_meter_lon
    half_extent_lat = half_extent * deg_per_meter_lat
    
    return [
        centroid_x - half_extent_lon,  # min_x
        centroid_y - half_extent_lat,  # min_y
        centroid_x + half_extent_lon,  # max_x
        centroid_y + half_extent_lat   # max_y
    ]

def split_large_polygon(geometry, patch_size=128, resolution=10):
    """Split a large polygon into multiple 128x128 patches"""
    from shapely.geometry import box
    
    bounds = geometry.bounds  # minx, miny, maxx, maxy
    centroid = geometry.centroid
    
    # Calculate polygon extent in meters
    extent_x_m = (bounds[2] - bounds[0]) * 111320 * np.cos(np.radians(centroid.y))
    extent_y_m = (bounds[3] - bounds[1]) * 110540
    
    patch_extent_m = patch_size * resolution  # 1280m for 128x128 at 10m
    
    # If polygon fits in one patch, return single centered patch
    if extent_x_m <= patch_extent_m and extent_y_m <= patch_extent_m:
        return [get_patch_bounds(centroid.x, centroid.y, patch_size, resolution)]
    
    # Otherwise, create grid of patches covering the polygon
    patches = []
    
    # Number of patches needed in each direction
    n_patches_x = int(np.ceil(extent_x_m / patch_extent_m))
    n_patches_y = int(np.ceil(extent_y_m / patch_extent_m))
    
    # Calculate step size in degrees
    deg_per_meter_lon = 1 / (111320 * np.cos(np.radians(centroid.y)))
    deg_per_meter_lat = 1 / 110540
    step_lon = patch_extent_m * deg_per_meter_lon
    step_lat = patch_extent_m * deg_per_meter_lat
    
    # Start from bottom-left of bounding box
    start_x = bounds[0] + step_lon / 2
    start_y = bounds[1] + step_lat / 2
    
    for i in range(n_patches_x):
        for j in range(n_patches_y):
            patch_center_x = start_x + i * step_lon
            patch_center_y = start_y + j * step_lat
            
            # Check if patch center is within or near the polygon
            patch_bounds = get_patch_bounds(patch_center_x, patch_center_y, patch_size, resolution)
            patch_box = box(patch_bounds[0], patch_bounds[1], patch_bounds[2], patch_bounds[3])
            
            if geometry.intersects(patch_box):
                patches.append(patch_bounds)
    
    return patches

# Test on first sample
test_geom = all_samples.iloc[0].geometry
test_patches = split_large_polygon(test_geom)
print(f"Sample polygon creates {len(test_patches)} patch(es)")
print(f"First patch bounds: {test_patches[0]}")

Sample polygon creates 40 patch(es)
First patch bounds: [np.float64(-0.8656314197659908), 34.971679731234644, np.float64(-0.8515983487718158), 34.983259249960895]


In [6]:
# Create all patches from all samples
all_patches = []

for idx, row in all_samples.iterrows():
    patches = split_large_polygon(row.geometry)
    for p_idx, patch_bounds in enumerate(patches):
        all_patches.append({
            'original_id': row['patch_id'],
            'patch_idx': p_idx,
            'class': row['class'],
            'split': row['split'],
            'bounds': patch_bounds,
            'center_lon': (patch_bounds[0] + patch_bounds[2]) / 2,
            'center_lat': (patch_bounds[1] + patch_bounds[3]) / 2
        })

patches_df = pd.DataFrame(all_patches)
patches_df['unique_id'] = range(len(patches_df))

print(f"Total patches to extract: {len(patches_df)}")
print(f"\nPatches by class:")
print(patches_df.groupby(['split', 'class']).size())
print(f"\nPatches by original sample:")
print(f"  Min patches per sample: {patches_df.groupby('original_id').size().min()}")
print(f"  Max patches per sample: {patches_df.groupby('original_id').size().max()}")
print(f"  Mean patches per sample: {patches_df.groupby('original_id').size().mean():.2f}")

Total patches to extract: 196

Patches by class:
split       class
train       0        62
            1        40
validation  0        53
            1        41
dtype: int64

Patches by original sample:
  Min patches per sample: 1
  Max patches per sample: 41
  Mean patches per sample: 1.68


In [7]:
# GEE extraction functions
def get_s2_median_composite(bounds, start_date, end_date, bands):
    """
    Get median composite of Sentinel-2 imagery for a given region and time period.
    Uses Sentinel-2 Surface Reflectance (SR) data with cloud masking.
    """
    # Create region geometry
    region = ee.Geometry.Rectangle(bounds)
    
    # Load Sentinel-2 SR collection
    s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(region) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    
    # Cloud masking function using SCL band
    def mask_clouds(image):
        scl = image.select('SCL')
        # Keep: vegetation (4), bare soil (5), water (6)
        # Mask: clouds (8,9,10), cloud shadow (3), snow (11)
        mask = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6))
        return image.updateMask(mask)
    
    # Apply cloud mask and compute median
    s2_masked = s2_sr.map(mask_clouds)
    median_composite = s2_masked.select(bands).median()
    
    return median_composite, region

def extract_patch_data(composite, region, patch_size=128, resolution=10):
    """
    Extract patch data as numpy array from GEE.
    Returns array of shape (bands, height, width).
    """
    # Sample the image at the region
    try:
        # Get the data as a numpy array via geemap
        data = geemap.ee_to_numpy(
            composite,
            region=region,
            scale=resolution,
            bands=S2_BANDS
        )
        
        if data is None:
            return None
            
        # Reshape to (bands, height, width) to match PASTIS format
        if len(data.shape) == 3:
            data = np.transpose(data, (2, 0, 1))
        
        # Ensure 128x128 size (crop or pad if necessary)
        if data.shape[1] != patch_size or data.shape[2] != patch_size:
            # Create padded/cropped array
            result = np.zeros((len(S2_BANDS), patch_size, patch_size), dtype=np.int16)
            h, w = min(data.shape[1], patch_size), min(data.shape[2], patch_size)
            result[:, :h, :w] = data[:, :h, :w]
            return result
            
        return data.astype(np.int16)
        
    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

print("GEE extraction functions defined successfully!")

GEE extraction functions defined successfully!


In [8]:
# Test extraction on one patch
test_bounds = patches_df.iloc[0]['bounds']
print(f"Testing extraction for patch with bounds: {test_bounds}")

# Get median composite
composite, region = get_s2_median_composite(test_bounds, START_DATE, END_DATE, S2_BANDS)

# Check image info
info = composite.getInfo()
print(f"\nComposite bands: {[b['id'] for b in info['bands']]}")

# Try to extract as numpy array using getDownloadURL method
try:
    # Alternative method: use ee.data.computePixels
    import requests
    from io import BytesIO
    
    # Get download URL for the region
    url = composite.getDownloadURL({
        'bands': S2_BANDS,
        'region': region,
        'scale': RESOLUTION,
        'format': 'NPY'
    })
    
    print(f"\nDownload URL obtained successfully!")
    print(f"Downloading test patch...")
    
    response = requests.get(url)
    if response.status_code == 200:
        test_data = np.load(BytesIO(response.content))
        print(f"Downloaded data shape: {test_data.shape}")
        print(f"Data type: {test_data.dtype}")
        print(f"Value range: {test_data.min()} to {test_data.max()}")
    else:
        print(f"Download failed: {response.status_code}")
        
except Exception as e:
    print(f"Error: {e}")

Testing extraction for patch with bounds: [np.float64(-0.8656314197659908), 34.971679731234644, np.float64(-0.8515983487718158), 34.983259249960895]

Composite bands: ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

Download URL obtained successfully!
Downloaded data shape: (130, 158)
Data type: [('B2', '<f8'), ('B3', '<f8'), ('B4', '<f8'), ('B5', '<f8'), ('B6', '<f8'), ('B7', '<f8'), ('B8', '<f8'), ('B8A', '<f8'), ('B11', '<f8'), ('B12', '<f8')]
Error: ufunc 'minimum' did not contain a loop with signature matching types (dtype([('B2', '<f8'), ('B3', '<f8'), ('B4', '<f8'), ('B5', '<f8'), ('B6', '<f8'), ('B7', '<f8'), ('B8', '<f8'), ('B8A', '<f8'), ('B11', '<f8'), ('B12', '<f8')]), dtype([('B2', '<f8'), ('B3', '<f8'), ('B4', '<f8'), ('B5', '<f8'), ('B6', '<f8'), ('B7', '<f8'), ('B8', '<f8'), ('B8A', '<f8'), ('B11', '<f8'), ('B12', '<f8')])) -> None


In [9]:
# Full extraction function with proper patch handling
import requests
from io import BytesIO
from tqdm import tqdm
import time

def download_patch(bounds, start_date, end_date, bands, resolution=10, patch_size=128):
    """
    Download a single patch from GEE as numpy array.
    Returns array of shape (bands, height, width) matching PASTIS format.
    """
    try:
        # Create region geometry
        region = ee.Geometry.Rectangle(bounds)
        
        # Load Sentinel-2 SR collection with cloud filtering
        s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(region) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
        
        # Cloud masking function
        def mask_clouds(image):
            scl = image.select('SCL')
            mask = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(2))  # veg, soil, water, dark
            return image.updateMask(mask)
        
        # Compute median composite
        s2_masked = s2_sr.map(mask_clouds)
        composite = s2_masked.select(bands).median()
        
        # Get download URL
        url = composite.getDownloadURL({
            'bands': bands,
            'region': region,
            'scale': resolution,
            'format': 'NPY'
        })
        
        # Download data
        response = requests.get(url, timeout=60)
        if response.status_code != 200:
            return None, f"HTTP {response.status_code}"
        
        # Load numpy array
        data = np.load(BytesIO(response.content))
        
        # Handle structured array (from GEE NPY format)
        if data.dtype.names is not None:
            # Convert structured array to regular array
            arrays = [data[name] for name in data.dtype.names]
            data = np.stack(arrays, axis=0)
        elif len(data.shape) == 3 and data.shape[2] == len(bands):
            # Transpose from (H, W, C) to (C, H, W)
            data = np.transpose(data, (2, 0, 1))
        
        # Ensure correct patch size
        n_bands = len(bands)
        result = np.zeros((n_bands, patch_size, patch_size), dtype=np.int16)
        
        h = min(data.shape[1], patch_size)
        w = min(data.shape[2], patch_size)
        
        # Center the data if smaller than patch_size
        start_h = (patch_size - h) // 2
        start_w = (patch_size - w) // 2
        
        result[:, start_h:start_h+h, start_w:start_w+w] = data[:, :h, :w].astype(np.int16)
        
        return result, None
        
    except Exception as e:
        return None, str(e)

print("Download function defined!")

Download function defined!


In [10]:
# Extract all patches and save to files
from datetime import datetime

# Create output directories
DATA_DIR = OUTPUT_DIR / 'DATA_S2'
ANNOTATIONS_DIR = OUTPUT_DIR / 'ANNOTATIONS'
DATA_DIR.mkdir(exist_ok=True)
ANNOTATIONS_DIR.mkdir(exist_ok=True)

# Track results
successful_downloads = []
failed_downloads = []

print(f"Starting extraction of {len(patches_df)} patches...")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Time period: {START_DATE} to {END_DATE}")
print("-" * 50)

for idx, row in tqdm(patches_df.iterrows(), total=len(patches_df), desc="Downloading patches"):
    patch_id = row['unique_id']
    bounds = row['bounds']
    
    # Download patch
    data, error = download_patch(
        bounds=bounds,
        start_date=START_DATE,
        end_date=END_DATE,
        bands=S2_BANDS,
        resolution=RESOLUTION,
        patch_size=PATCH_SIZE
    )
    
    if data is not None:
        # Save to .npy file (matching PASTIS format)
        # PASTIS format: (Time, Bands, Height, Width) - we have single time, so (1, 10, 128, 128)
        data_with_time = data[np.newaxis, :, :, :]  # Add time dimension
        
        filename = f"S2_{patch_id:05d}.npy"
        np.save(DATA_DIR / filename, data_with_time)
        
        successful_downloads.append({
            'patch_id': patch_id,
            'original_id': row['original_id'],
            'class': row['class'],
            'split': row['split'],
            'center_lon': row['center_lon'],
            'center_lat': row['center_lat'],
            'filename': filename
        })
    else:
        failed_downloads.append({
            'patch_id': patch_id,
            'error': error
        })
    
    # Small delay to avoid rate limiting
    time.sleep(0.5)

print(f"\n{'='*50}")
print(f"Extraction complete!")
print(f"  Successful: {len(successful_downloads)}")
print(f"  Failed: {len(failed_downloads)}")

Starting extraction of 196 patches...
Output directory: /home/crop/Desktop/crop2/output/algeria_s2_data
Time period: 2023-01-01 to 2023-04-30
--------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# Save metadata and annotations (matching PASTIS format)
import json

# Create metadata DataFrame
metadata_df = pd.DataFrame(successful_downloads)

# Save as GeoJSON (similar to PASTIS metadata.geojson)
if len(successful_downloads) > 0:
    metadata_gdf = gpd.GeoDataFrame(
        metadata_df,
        geometry=gpd.points_from_xy(metadata_df['center_lon'], metadata_df['center_lat']),
        crs='EPSG:4326'
    )
    metadata_gdf.to_file(OUTPUT_DIR / 'metadata.geojson', driver='GeoJSON')
    
    # Save class labels for each patch (ANNOTATIONS)
    for _, row in metadata_df.iterrows():
        patch_id = row['patch_id']
        class_label = row['class']
        # Save as single value array (like ParcelIDs in PASTIS)
        np.save(ANNOTATIONS_DIR / f'Labels_{patch_id:05d}.npy', np.array([class_label]))
    
    # Save summary JSON
    summary = {
        'extraction_date': datetime.now().isoformat(),
        'source': 'Sentinel-2 SR Harmonized (GEE)',
        'time_range': {'start': START_DATE, 'end': END_DATE},
        'bands': S2_BANDS,
        'resolution_m': RESOLUTION,
        'patch_size': PATCH_SIZE,
        'total_patches': len(successful_downloads),
        'classes': {'0': 'non-cereal', '1': 'cereal'},
        'splits': metadata_df['split'].value_counts().to_dict(),
        'class_distribution': metadata_df['class'].value_counts().to_dict()
    }
    
    with open(OUTPUT_DIR / 'extraction_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print("Metadata and annotations saved!")
    print(f"\nDataset Summary:")
    print(f"  - Total patches: {len(successful_downloads)}")
    print(f"  - Training patches: {len(metadata_df[metadata_df['split']=='train'])}")
    print(f"  - Validation patches: {len(metadata_df[metadata_df['split']=='validation'])}")
    print(f"  - Cereal patches: {len(metadata_df[metadata_df['class']==1])}")
    print(f"  - Non-cereal patches: {len(metadata_df[metadata_df['class']==0])}")
else:
    print("No successful downloads to save metadata for.")

In [ ]:
# Visualize some extracted patches
import matplotlib.pyplot as plt

# Load and visualize first few patches
if len(successful_downloads) > 0:
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i, row in enumerate(successful_downloads[:8]):
        patch_file = DATA_DIR / row['filename']
        data = np.load(patch_file)  # Shape: (1, 10, 128, 128)
        
        # Create RGB composite (B4, B3, B2 = indices 2, 1, 0)
        rgb = data[0, [2, 1, 0], :, :]  # (3, 128, 128)
        rgb = np.transpose(rgb, (1, 2, 0))  # (128, 128, 3)
        
        # Normalize for display
        rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8)
        rgb = np.clip(rgb * 2.5, 0, 1)  # Brightness adjustment
        
        ax = axes[i]
        ax.imshow(rgb)
        class_name = 'Cereal' if row['class'] == 1 else 'Non-Cereal'
        ax.set_title(f"Patch {row['patch_id']} ({class_name})\n{row['split']}")
        ax.axis('off')
    
    # Hide empty subplots
    for i in range(len(successful_downloads[:8]), 8):
        axes[i].axis('off')
    
    plt.suptitle('Extracted Sentinel-2 Patches (RGB: B4-B3-B2)', fontsize=14)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'extracted_patches_preview.png', dpi=150)
    plt.show()
    
    print(f"\nPreview saved to: {OUTPUT_DIR / 'extracted_patches_preview.png'}")
else:
    print("No patches to visualize - run extraction first.")